In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/6.vehicle'):
    shutil.rmtree('./file/6.vehicle')

os.makedirs('./file/6.vehicle')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/5.oem'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/5.oem/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        request_url, check_url = f'''https://productdesk-api.cellacore.net/legacy/cardone/Vehicle/Recommendation/?q=(productSKU.keyword:{input_.loc[a, 'Part Number']})&size=5000&useCache=true&auth={input_.loc[a, 'Auth']}''', input_.loc[a, 'Url']
                        
                        resp = requests.get(request_url,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                list_dict = resp.json()

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame()
                for i, dict_ in enumerate(list_dict):
                    df_temp_temp, list_key = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                                            'Part Number': input_.loc[a, 'Part Number'],
                                                            'Vehicle No': i + 1}]), []
                
                    for key, value in dict_['fitment']['Vcdb'].items():
                        if type(value) in [int, float]:
                            df_temp_temp.loc[0, key.strip()] = value
                        elif type(value) == str:
                            df_temp_temp.loc[0, key.strip()] = value.strip()
                        elif type(value) == list:
                            list_key.append(key)
                        else:
                            raise
                
                    for key in list_key:
                        list_df_temp_temp_temp = []
                        for j in range(len(df_temp_temp)):
                            df_temp_temp_temp = pd.DataFrame([df_temp_temp.iloc[j]]).reset_index(drop=True)
                            for k in range(len(dict_['fitment']['Vcdb'][key])):
                                for key_2, value_2 in dict_['fitment']['Vcdb'][key][k].items():
                                    if type(value_2) in [int, float]:
                                        df_temp_temp_temp.loc[0, f'[{key.strip()}]-{key_2.strip()}'] = value_2
                                    elif type(value_2) == str:
                                        df_temp_temp_temp.loc[0, f'[{key.strip()}]-{key_2.strip()}'] = value_2.strip()
                                    else:
                                        raise
                            list_df_temp_temp_temp.append(df_temp_temp_temp)
                        df_temp_temp = pd.concat(list_df_temp_temp_temp, ignore_index=True).fillna('')
                
                    df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                if df_temp.empty:
                    raise

                # = = = = = = = = = = = = = = =
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
                df_temp.loc[0, 'Request_Url'] = request_url
                df_temp.loc[0, 'Check_Url'] = check_url
                
                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{request_url}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        list_column = output_correct.columns[3:]
        output_correct = output_correct[['No', 'Part Number', 'Vehicle No'] + sorted(list_column)].sort_values(by=['No', 'Vehicle No'],
                                                                                                               ascending=[True, True],
                                                                                                               ignore_index=True)
        output_correct.to_excel(f'''./file/6.vehicle/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-vehicle_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/6.vehicle'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/6.vehicle/{file}',
              f'''./file/6.vehicle/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '-vehicle.xlsx', file)}''')

print('Done ~')

总数量：10

[状态：error，尝试次数：1] - 50.00% - crawler_1 > 2.https://productdesk-api.cellacore.net/legacy/cardone/Vehicle/Recommendation/?q=(productSKU.keyword:40-10000)&size=5000&useCache=true&auth=mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw
[剩余数量：5] - [当前时间：14:01:37]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 5.https://productdesk-api.cellacore.net/legacy/cardone/Vehicle/Recommendation/?q=(productSKU.keyword:40-10007)&size=5000&useCache=true&auth=mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw
[剩余数量：4] - [当前时间：14:01:39]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 1.https://productdesk-api.cellacore.net/legacy/cardone/Vehicle/Recommendation/?q=(productSKU.keyword:40-1000)&size=5000&useCache=true&auth=mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw
[剩余数量：3] - [当前时间：14:01:42]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 4.https://productdesk-api.cellacore.net/legacy/cardone/Vehicle/Recommendation/?q=(productSKU.keyword:40-10006)&size=5000&useCache=true&auth=mXWIhUuYvk3hklMcdwLi4lwVd3s6VASw
[剩余数量：2] - [当前时间：14:01:45]

[状态：ok，尝试次数：1] - 50.00% - crawler_

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 3782.06it/s]

Done ~
